In [1]:
import requests
import pandas as pd
import numpy as np
import os
import json
import warnings
warnings.filterwarnings("ignore")
base_uri = 'https://uts-ws.nlm.nih.gov'
apikey = "4e2f94ae-3ff5-4948-92f7-bf59c986d663" #API key goes here

In [7]:
hypertension_df = pd.read_csv("hypertension_5771.csv")
hypertension_df.drop('Unnamed: 0',axis=1,inplace=True)
hypertension_df.head()

,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,mfr_num,...,dose_form,dose_freq,indi_pt,pt,auth_num,lit_ref,age_grp,sex,prod_ai,drug_rec_act
0,57419732,5741973,2,F,20041114.0,20130314.0,20050218,20130321,EXP,GB-JNJFOC-20050203627,...,NaN,QD,Hypertension,Barrett's oesophagus,NaN,NaN,NaN,NaN,NaN,NaN
1,57419732,5741973,2,F,20041114.0,20130314.0,20050218,20130321,EXP,GB-JNJFOC-20050203627,...,NaN,QD,Hypertension,Haematemesis,NaN,NaN,NaN,NaN,NaN,NaN
2,57571695,5757169,5,F,2005.0,20130307.0,20050302,20130318,EXP,2005BI003595,...,NaN,NaN,Hypertension,Dyspnoea,NaN,NaN,NaN,NaN,NaN,NaN
3,57571695,5757169,5,F,2005.0,20130307.0,20050302,20130318,EXP,2005BI003595,...,NaN,NaN,Hypertension,Hallucination,NaN,NaN,NaN,NaN,NaN,NaN
4,57571695,5757169,5,F,2005.0,20130307.0,20050302,20130318,EXP,2005BI003595,...,NaN,NaN,Hypertension,Impaired healing,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
hypertension_df['drugname'] = hypertension_df['drugname'].str.strip().str.lower()

In [9]:
hypertension_drugname = hypertension_df["drugname"].unique() #get unique drug names from the dataset 
hypertension_drug = pd.DataFrame(hypertension_drugname, columns = ["drugname"]) #put the drug names in a dataframe

In [10]:
hypertension_drug

,drugname
0,atenolol
1,hydralazine
2,verapamil
3,furosemide
4,hydrochlorothiazide
5,felodipine
6,doxazosin
7,amlodipine
8,ramipril
9,methyldopa


In [11]:
plhold = [] #create an empty list to hold the name in UMLS API search result for the drug name normalization
uihold = [] #create an empty list to hold the cui in UMLS API search result for the drug name normalization

In [23]:
#loop through the hypertension data to search all drug names
for i in range(0,len(hypertension_drug)):
    path = '/rest/search/current'
    string = hypertension_drug['drugname'][i]
    query = {'apiKey':apikey, 'string':string, "sabs": "RXNORM"}
    output = requests.get(base_uri+path, params=query)
    output.encoding = 'utf-8'
    outputJson = output.json()
    
    #if no result is found in the search,'N/A' is appended in the list for UMLS name and cui
    if not 'result' in outputJson or len(outputJson['result']['results']) == 0: 
        path = '/rest/search/current'
        query = {'apiKey':apikey, 'string':string}
        output = requests.get(base_uri+path, params=query )
        output.encoding = 'utf-8'
        outputJson = output.json()
        if not 'result' in outputJson or len(outputJson['result']['results']) == 0: 
            plhold.append('N/A')
            uihold.append('N/A')
        else:
            #else append the first result of the search to the list.
            plhold.append(outputJson['result']['results'][0]['name'])
            uihold.append(outputJson['result']['results'][0]['ui'])
            
    else:
        #else append the first result of the search to the list.
        plhold.append(outputJson['result']['results'][0]['name'])
        uihold.append(outputJson['result']['results'][0]['ui'])

In [24]:
#Create a dataframe with the UMLS name, cui and the unique drug name from the ADHD data
UMLS_df = pd.DataFrame({'UMLS_Name': plhold, 'UMLS_name_UI':uihold})#Create a dataframe with the UMLS name, cui
UMLS_df['drugname'] = hypertension_drug #append the drugname column from the ADHD data to the newly created dataframe.

In [25]:
UMLS_df

,UMLS_Name,UMLS_name_UI,drugname
0,atenolol,C0004147,atenolol
1,hydralazine,C0020223,hydralazine
2,verapamil,C0042523,verapamil
3,furosemide,C0016860,furosemide
4,hydrochlorothiazide,C0020261,hydrochlorothiazide
5,felodipine,C0015772,felodipine
6,doxazosin,C0114873,doxazosin
7,amlodipine,C0051696,amlodipine
8,ramipril,C0072973,ramipril
9,methyldopa,C0025741,methyldopa


In [27]:
UMLS_df.to_csv('UMLS_ht_drugname.csv', index=False)

In [5]:
#Repeat the process to get details for drug reaction
unique_pt = hypertension_df["pt"].unique() #create a list of unique drug reactions
pt_reac = pd.DataFrame(unique_pt, columns = ['pt']) #transform the list to a dataframe
pt_hold = [] #empty list to UMLS drug reaction standard name
pt_ui = [] #empty list to UMLS drug reaction cui

In [12]:
for i in range(0,len(pt_reac)):
    path = '/rest/search/current'
    string = pt_reac['pt'][i]
    query = {'apiKey':apikey, 'string':string, "sabs": "MDR"}
    output = requests.get(base_uri+path, params=query)
    output.encoding = 'utf-8'
    outputJson = output.json()
    if not 'result' in outputJson or len(outputJson['result']['results']) == 0:
        path = '/rest/search/current'
        query = {'apiKey':apikey, 'string':string}
        output = requests.get(base_uri+path, params=query )
        output.encoding = 'utf-8'
        outputJson = output.json()
        if not 'result' in outputJson or len(outputJson['result']['results']) == 0: 
            pt_hold.append('N/A')
            pt_ui.append('N/A')
        else:
            #else append the first result of the search to the list.
            plhold.append(outputJson['result']['results'][0]['name'])
            uihold.append(outputJson['result']['results'][0]['ui'])
    else:
        pt_hold.append(outputJson['result']['results'][0]['name'])
        pt_ui.append(outputJson['result']['results'][0]['ui'])

In [13]:
pt_df = pd.DataFrame({'UMLS_pt': pt_hold, 'UMLS_pt_UI': pt_ui}) #create a dataframe from the list of drug reaction names/cui from UMLS
pt_df['pt'] = pt_reac['pt'] #append the drug reaction column from the ADHD data to the newly created dataframe.

In [14]:
pt_df

,UMLS_pt,UMLS_pt_UI,pt
0,Hematemesis,C0018926,Barrett's oesophagus
1,Dyspnea,C0013404,Haematemesis
2,Hallucinations,C0018524,Dyspnoea
3,Impaired wound healing,C0151692,Hallucination
4,Multiple sclerosis relapse,C0856120,Impaired healing
...,...,...,...
7986,Administration site wound,C5243740,Eye symptom
7987,ATRIAL ENLARGEMENT,C0741276,Administration site wound
7988,Foreign body in reproductive tract,C4523983,Atrial enlargement
7989,Hypomenorrhea,C0020624,Foreign body in reproductive tract


In [15]:
pt_df.to_csv('UMLS_ht_pt.csv', index=False) 

In [16]:
#Import required libraries
import pandas as pd

#Read in drug reaction reference file
norm_pt = pd.read_csv('UMLS_ht_pt.csv')
norm_pt.head(5)

,UMLS_pt,UMLS_pt_UI,pt
0,Hematemesis,C0018926,Barrett's oesophagus
1,Dyspnea,C0013404,Haematemesis
2,Hallucinations,C0018524,Dyspnoea
3,Impaired wound healing,C0151692,Hallucination
4,Multiple sclerosis relapse,C0856120,Impaired healing


In [17]:
#create a dictionary with the drug reaction name and normalized CUI for pt. Set the 'pt' column as index
pt_dict = norm_pt.to_dict()
pt_dict['UMLS_pt']

{0: 'Hematemesis',
 1: 'Dyspnea',
 2: 'Hallucinations',
 3: 'Impaired wound healing',
 4: 'Multiple sclerosis relapse',
 5: 'Myocardial Infarction',
 6: 'Postoperative wound complication',
 7: 'Surgical Wound Infection',
 8: 'Wound closure',
 9: 'Wound dehiscence',
 10: 'Wound evisceration',
 11: 'Abdominal discomfort',
 12: 'Abdominal Pain',
 13: 'Abscess of jaw',
 14: 'Actinomycosis',
 15: 'Agitation',
 16: 'Anemia',
 17: 'Anxiety',
 18: 'Aortic valve calcification',
 19: 'Arteriosclerosis',
 20: 'Arthralgia',
 21: 'Arthritis',
 22: 'Arthropathy',
 23: 'Atelectasis',
 24: 'Avulsion fracture',
 25: 'Back Pain',
 26: 'Beta haemolytic streptococcal infection',
 27: 'Bipolar Disorder',
 28: 'Bone Diseases',
 29: 'Bone pain',
 30: 'Breast tenderness',
 31: 'Cardiac Arrest',
 32: 'Heart murmur',
 33: 'Internal Carotid Artery Stenosis',
 34: 'Carpal Tunnel Syndrome',
 35: 'Impacted cerumen',
 36: 'Radiculopathy, Cervical',
 37: 'Cholecystitis',
 38: 'Cholelithiasis',
 39: 'Chronic Obstructi

In [18]:
# Create a dictionary using pt as key and CUI as value
norm_pt_dict = norm_pt.set_index('pt').to_dict()['UMLS_pt']
norm_pt_dict

{"Barrett's oesophagus": 'Hematemesis',
 'Haematemesis': 'Dyspnea',
 'Dyspnoea': 'Hallucinations',
 'Hallucination': 'Impaired wound healing',
 'Impaired healing': 'Multiple sclerosis relapse',
 'Multiple sclerosis relapse': 'Myocardial Infarction',
 'Myocardial infarction': 'Postoperative wound complication',
 'Postoperative wound complication': 'Surgical Wound Infection',
 'Postoperative wound infection': 'Wound closure',
 'Wound closure': 'Wound dehiscence',
 'Wound dehiscence': 'Wound evisceration',
 'Wound evisceration': 'Abdominal discomfort',
 'Abdominal discomfort': 'Abdominal Pain',
 'Abdominal pain': 'Abscess of jaw',
 'Abscess jaw': 'Actinomycosis',
 'Actinomycosis': 'Agitation',
 'Agitation': 'Anemia',
 'Anaemia': 'Anxiety',
 'Anxiety': 'Aortic valve calcification',
 'Aortic calcification': 'Arteriosclerosis',
 'Arteriosclerosis': 'Arthralgia',
 'Arthralgia': 'Arthritis',
 'Arthritis': 'Arthropathy',
 'Arthropathy': 'Atelectasis',
 'Atelectasis': 'Avulsion fracture',
 'Avul

In [23]:
#Read in hypertension file
hypertension_data = pd.read_csv("hypertension_5771.csv", low_memory=False)
hypertension_data.drop('Unnamed: 0',axis=1,inplace=True)
hypertension_data['drugname'] = hypertension_data['drugname'].str.strip().str.lower()
hypertension_data

,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,mfr_num,...,dose_form,dose_freq,indi_pt,pt,auth_num,lit_ref,age_grp,sex,prod_ai,drug_rec_act
0,57419732,5741973,2,F,20041114.0,20130314.0,20050218,20130321,EXP,GB-JNJFOC-20050203627,...,NaN,QD,Hypertension,Barrett's oesophagus,NaN,NaN,NaN,NaN,NaN,NaN
1,57419732,5741973,2,F,20041114.0,20130314.0,20050218,20130321,EXP,GB-JNJFOC-20050203627,...,NaN,QD,Hypertension,Haematemesis,NaN,NaN,NaN,NaN,NaN,NaN
2,57571695,5757169,5,F,2005.0,20130307.0,20050302,20130318,EXP,2005BI003595,...,NaN,NaN,Hypertension,Dyspnoea,NaN,NaN,NaN,NaN,NaN,NaN
3,57571695,5757169,5,F,2005.0,20130307.0,20050302,20130318,EXP,2005BI003595,...,NaN,NaN,Hypertension,Hallucination,NaN,NaN,NaN,NaN,NaN,NaN
4,57571695,5757169,5,F,2005.0,20130307.0,20050302,20130318,EXP,2005BI003595,...,NaN,NaN,Hypertension,Impaired healing,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428964,94935878,9493587,8,F,2008.0,20201214.0,20130902,20201217,PER,US-PFIZER INC-2013252347,...,NaN,NaN,Hypertension,Drug ineffective,NaN,NaN,NaN,F,VERAPAMIL HYDROCHLORIDE,NaN
428965,94935878,9493587,8,F,2008.0,20201214.0,20130902,20201217,PER,US-PFIZER INC-2013252347,...,NaN,NaN,Hypertension,Maternal exposure during pregnancy,NaN,NaN,NaN,F,VERAPAMIL HYDROCHLORIDE,NaN
428966,99193863,9919386,3,F,2005.0,20201116.0,20140224,20201118,EXP,US-PFIZER INC-2014050633,...,NaN,QD,Hypertension,Nephropathy,NaN,NaN,NaN,F,AMLODIPINE BESYLATE,NaN
428967,99193863,9919386,3,F,2005.0,20201116.0,20140224,20201118,EXP,US-PFIZER INC-2014050633,...,NaN,QD,Hypertension,Neuropathy peripheral,NaN,NaN,NaN,F,AMLODIPINE BESYLATE,NaN


In [28]:
#Create a dataframe for ADHD drug and reaction containing drugname and reaction columns from the ADHD data
hypertension_drugNreac = hypertension_data[['primaryid', 'caseid', 'drugname', 'pt','age']] 

#Loop through the dataset to assign drug reaction CUI using 'pt' column as reference
hypertension_drugNreac['UMLS_pt'] = hypertension_drugNreac['pt'].map(norm_pt_dict)
hypertension_drugNreac.head(5)

,primaryid,caseid,drugname,pt,age,UMLS_pt
0,57419732,5741973,atenolol,Barrett's oesophagus,77.0,Hematemesis
1,57419732,5741973,atenolol,Haematemesis,77.0,Dyspnea
2,57571695,5757169,hydralazine,Dyspnoea,52.0,Hallucinations
3,57571695,5757169,hydralazine,Hallucination,52.0,Impaired wound healing
4,57571695,5757169,hydralazine,Impaired healing,52.0,Multiple sclerosis relapse


In [29]:
#Read in drug name reference file
drugname_norm = pd.read_csv('UMLS_ht_drugname.csv')
drugname_norm.head(5)

,UMLS_Name,UMLS_name_UI,drugname
0,atenolol,C0004147,atenolol
1,hydralazine,C0020223,hydralazine
2,verapamil,C0042523,verapamil
3,furosemide,C0016860,furosemide
4,hydrochlorothiazide,C0020261,hydrochlorothiazide


In [30]:
#Create a dataframe for ADHD drug name and CUI
drugname_norm_data = drugname_norm[['UMLS_Name', 'drugname']]

#Set drugname as index in reference file
drugname_norm_dict = drugname_norm_data.set_index('drugname').to_dict()['UMLS_Name']

#Map drugname CUI to dataframe earlier created for ADHD drug and reaction
hypertension_drugNreac['UMLS_drugname'] = hypertension_drugNreac['drugname'].map(drugname_norm_dict)
hypertension_drugNreac.head(5)

,primaryid,caseid,drugname,pt,age,UMLS_pt,UMLS_drugname
0,57419732,5741973,atenolol,Barrett's oesophagus,77.0,Hematemesis,atenolol
1,57419732,5741973,atenolol,Haematemesis,77.0,Dyspnea,atenolol
2,57571695,5757169,hydralazine,Dyspnoea,52.0,Hallucinations,hydralazine
3,57571695,5757169,hydralazine,Hallucination,52.0,Impaired wound healing,hydralazine
4,57571695,5757169,hydralazine,Impaired healing,52.0,Multiple sclerosis relapse,hydralazine


In [31]:
#Resulting file is saved to file
hypertension_drugNreac.to_csv('hypertension_norm_data.csv', index=False)